In [1]:
import numpy as np
#import cupy as cp
import matplotlib.pyplot as plt
from scipy import stats

MHD-PIC : S. Usami et al., 2013 スタイル

PICパート

In [2]:
def get_rho(q_list, x, n_x, dx):
    x_index = np.floor(x[0, :] / dx).astype(np.int64)
    rho = np.zeros([n_x])

    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    index_one_array = x_index

    rho += np.bincount(index_one_array, 
                       weights=q_list * cx2,
                       minlength=n_x
                      )
    rho += np.roll(np.bincount(index_one_array, 
                               weights=q_list * cx1,
                               minlength=n_x
                              ), 1, axis=0)
    
    return rho


def solve_poisson_not_periodic(rho, n_x, epsilon0, dx, E):
    phi = np.zeros(n_x)
    for k in range(10000):
        phi = (rho/epsilon0 * dx**2 + np.roll(phi, -1) + np.roll(phi, 1)) / 2.0
    E[0, :] = -(np.roll(phi, -1) - phi) / dx
    return E


def E_modification(q_list, x, n_x, dx, epsilon0, E):
    rho = get_rho(q_list, x, n_x, dx)
    div_E = (E[0, :] - np.roll(E[0, :], 1)) / dx
    delta_rho = rho - div_E

    delta_E = np.zeros(E.shape)
    delta_E = solve_poisson_not_periodic(delta_rho, n_x, epsilon0, dx, delta_E)
    E += delta_E
    
    return E 


def get_current_density(c, q_list, v, x, n_x, dx, dt):
    x_index = np.floor(x[0, :] / dx).astype(int)
    x_index_half = np.floor((x[0, :] - 1/2*dx) / dx).astype(int)
    x_index_half_minus = np.where(x_index_half == -1)
    x_index_half[x_index_half == -1] = n_x-1

    current = np.zeros([3, n_x])
    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)


    cx1 = (x[0, :] - (x_index_half + 1/2)*dx)/dx  
    cx2 = ((x_index_half + 3/2)*dx - x[0, :])/dx 
    cx1[x_index_half_minus] = (x[0, x_index_half_minus] - (-1/2)*dx)/dx  
    cx2[x_index_half_minus] = ((1/2)*dx - x[0, x_index_half_minus])/dx 
    index_one_array = x_index_half

    current[0, :] += np.bincount(index_one_array, 
                                 weights=q_list * v[0, :]/gamma * cx2, 
                                 minlength=n_x
                                )
    current[0, :] += np.roll(np.bincount(index_one_array, 
                                         weights=q_list * v[0, :]/gamma * cx1, 
                                         minlength=n_x
                                        ), 1, axis=0)


    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    index_one_array = x_index
    
    current[1, :] += np.bincount(index_one_array, 
                                    weights=q_list * v[1, :]/gamma * cx2, 
                                    minlength=n_x
                                    )
    current[1, :] += np.roll(np.bincount(index_one_array, 
                                            weights=q_list * v[1, :]/gamma * cx1, 
                                            minlength=n_x
                                            ), 1, axis=0)

    current[2, :] += np.bincount(index_one_array, 
                                 weights=q_list * v[2, :]/gamma * cx2, 
                                 minlength=n_x
                                )
    current[2, :] += np.roll(np.bincount(index_one_array, 
                                         weights=q_list * v[2, :]/gamma * cx1, 
                                         minlength=n_x
                                        ), 1, axis=0)
    
    return current


def buneman_boris_v(c, dt, q_list, m_list, E, B, v):

    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)

    #TとSの設定
    T = (q_list/m_list) * dt * B / 2.0 / gamma
    S = 2.0 * T / (1.0 + np.linalg.norm(T, axis=0)**2)

    #時間発展
    v_minus = v + (q_list/m_list) * E * (dt/2)
    v_0 = v_minus + np.cross(v_minus, T, axis=0)
    v_plus = v_minus + np.cross(v_0, S, axis=0)
    v = v_plus + (q_list/m_list) * E * (dt/2.0)

    return v 


def buneman_boris_x(c, dt, v, x):

    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)

    x = x + v * dt / gamma

    return x


def time_evolution_v(c, E, B, x, q_list, m_list, n_x, dx, dt, v):
    E_tmp = E.copy()
    B_tmp = B.copy()
    E_tmp[0, :] = (E[0, :] + np.roll(E[0, :], 1, axis=0)) / 2.0
    B_tmp[0, :] = (B[0, :] + np.roll(B[0, :], -1, axis=0)) / 2.0
    x_index = np.floor(x[0, :] / dx).astype(int)
    x_index_half = np.floor((x[0, :] - 1/2*dx) / dx).astype(int)
    x_index_half_minus = np.where(x_index_half == -1)
    x_index_half[x_index_half == -1] = n_x-1
    E_particle = np.zeros(x.shape)
    B_particle = np.zeros(x.shape)

    #電場
    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    cx1 = cx1.reshape(-1, 1)
    cx2 = cx2.reshape(-1, 1)
    E_particle[:, :] = (E_tmp[:, x_index].T * cx2 \
                     + E_tmp[:, (x_index+1)%n_x].T * cx1 \
                    ).T
    
    #磁場
    cx1 = (x[0, :] - (x_index_half + 1/2)*dx)/dx  
    cx2 = ((x_index_half + 3/2)*dx - x[0, :])/dx 
    cx1[x_index_half_minus] = (x[0, x_index_half_minus] - (-1/2)*dx)/dx  
    cx2[x_index_half_minus] = ((1/2)*dx - x[0, x_index_half_minus])/dx 
    cx1 = cx1.reshape(-1, 1)
    cx2 = cx2.reshape(-1, 1)
    B_particle[:, :] = (B_tmp[:, x_index_half].T * cx2 \
                     + B_tmp[:, (x_index_half+1)%n_x].T * cx1 \
                    ).T
  
    v = buneman_boris_v(c, dt, q_list, m_list, E_particle, B_particle, v)

    return v


def time_evolution_x(c, dt, v, x):
    
    x = buneman_boris_x(c, dt, v, x)

    return x 


def time_evolution_E(B, current, c, epsilon0, dx, dt, E):
    E[0, :] = -current[0, :]/epsilon0 * dt + E[0, :]
    E[1, :] = (-current[1, :]/epsilon0 \
            - c**2 * (B[2, :] - np.roll(B[2, :], 1))/dx) * dt \
            + E[1, :]
    E[2, :] = (-current[2, :]/epsilon0 \
            + c**2 * (B[1, :] - np.roll(B[1, :], 1)) / dx) * dt \
            + E[2, :]
    return E


def time_evolution_B(E, dx, dt, B):
    B[0, :] = B[0, :]
    B[1, :] = -(-(np.roll(E[2, :], -1) - E[2, :]) / dx) * dt \
            + B[1, :]
    B[2, :] = -((np.roll(E[1, :], -1) - E[1, :]) / dx) * dt \
            + B[2, :]
    return B


def periodic_condition_x(x, x_max):

    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = 1e-10

    under_x0_index = np.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = x_max - 1e-10

    return x 


def refrective_condition_x(v, x, x_max):

    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = x_max - 1e-10
    v[0, over_xmax_index] = -v[0, over_xmax_index]

    under_x0_index = np.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = 1e-10
    v[0, under_x0_index] = -v[0, under_x0_index]

    return v, x


def open_condition_x(v_past, x_past, x_min, x_max, dx, v, x):

    first_inf_index = np.argmax(v[0, :] == np.inf) #v[0]でなくてもOK
    new_particle_x0_index = np.where((x_past[0, : ] < x_min + dx) & (x[0, : ] >= x_min + dx))
    v[:, first_inf_index:first_inf_index+new_particle_x0_index.sum()] = v[:, new_particle_x0_index]
    x[:, first_inf_index:first_inf_index+new_particle_x0_index.sum()] = x[:, new_particle_x0_index]

    first_inf_index += new_particle_x0_index.sum()
    new_particle_xmax_index = np.where((x_past[0, : ] > x_max - dx) & (x[0, : ] <= x_max - dx))
    v[:, first_inf_index:first_inf_index+new_particle_xmax_index.sum()] = v[:, new_particle_xmax_index]
    x[:, first_inf_index:first_inf_index+new_particle_xmax_index.sum()] = x[:, new_particle_xmax_index]


    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    v[:, over_xmax_index] = np.inf
    x[:, over_xmax_index] = np.inf

    under_x0_index = np.where(x[0, :] <= x_min)[0]
    v[:, under_x0_index] = np.inf
    x[:, under_x0_index] = np.inf

    v, x = sort_particles(v, x)

    return v, x


def sort_particles(v, x):

    exist_particles_index = np.isfinite(v[0, :]) #v[0]でなくてもOK
    v = np.concatenate(
        (v[:, exist_particles_index], 
         [np.inf] * (v.shape[1] - exist_particles_index.sum())
        ))
    x = np.concatenate(
        (x[:, exist_particles_index], 
         [np.inf] * (x.shape[1] - exist_particles_index.sum())
        ))
    
    return v, x

理想MHDパート

In [3]:
def minmod(x, y):
    return np.sign(x) * np.maximum(np.minimum(np.abs(x), np.sign(x) * y), 1e-20)


def get_L_component(rho, u, v, w, By, Bz, p, axis):
    rho_L = rho + 1/2 * minmod(rho - np.roll(rho, 1, axis=axis), np.roll(rho, -1, axis=axis) - rho)
    u_L = u + 1/2 * minmod(u - np.roll(u, 1, axis=axis), np.roll(u, -1, axis=axis) - u)
    v_L = v + 1/2 * minmod(v - np.roll(v, 1, axis=axis), np.roll(v, -1, axis=axis) - v)
    w_L = w + 1/2 * minmod(w - np.roll(w, 1, axis=axis), np.roll(w, -1, axis=axis) - w)
    By_L = By + 1/2 * minmod(By - np.roll(By, 1, axis=axis), np.roll(By, -1, axis=axis) - By)
    Bz_L = Bz + 1/2 * minmod(Bz - np.roll(Bz, 1, axis=axis), np.roll(Bz, -1, axis=axis) - Bz)
    p_L = p + 1/2 * minmod(p - np.roll(p, 1, axis=axis), np.roll(p, -1, axis=axis) - p)

    return rho_L, u_L, v_L, w_L, By_L, Bz_L, p_L


def get_R_component(rho, u, v, w, By, Bz, p, axis):
    rho_R = np.roll(rho, -1, axis=axis) - 1/2 * minmod(np.roll(rho, -1, axis=axis) - rho, np.roll(rho, -2, axis=axis) - np.roll(rho, -1, axis=axis))
    u_R = np.roll(u, -1, axis=axis) - 1/2 * minmod(np.roll(u, -1, axis=axis) - u, np.roll(u, -2, axis=axis) - np.roll(u, -1, axis=axis))
    v_R = np.roll(v, -1, axis=axis) - 1/2 * minmod(np.roll(v, -1, axis=axis) - v, np.roll(v, -2, axis=axis) - np.roll(v, -1, axis=axis))
    w_R = np.roll(w, -1, axis=axis) - 1/2 * minmod(np.roll(w, -1, axis=axis) - w, np.roll(w, -2, axis=axis) - np.roll(w, -1, axis=axis))
    By_R = np.roll(By, -1, axis=axis) - 1/2 * minmod(np.roll(By, -1, axis=axis) - By, np.roll(By, -2, axis=axis) - np.roll(By, -1, axis=axis))
    Bz_R = np.roll(Bz, -1, axis=axis) - 1/2 * minmod(np.roll(Bz, -1, axis=axis) - Bz, np.roll(Bz, -2, axis=axis) - np.roll(Bz, -1, axis=axis))
    p_R = np.roll(p, -1, axis=axis) - 1/2 * minmod(np.roll(p, -1, axis=axis) - p, np.roll(p, -2, axis=axis) - np.roll(p, -1, axis=axis))
    
    return rho_R, u_R, v_R, w_R, By_R, Bz_R, p_R


def get_U_parameters_1(rho, u, v, w, Bx, By, Bz, e, pT, pT1, S, SM):
    rho1 = rho * (S - u) / (S - SM)
    u1 = SM
    v1 = v - Bx * By * (SM - u) / (rho * (S - u) * (S - SM) - Bx**2)
    w1 = w - Bx * Bz * (SM - u) / (rho * (S - u) * (S - SM) - Bx**2)
    By1 = By * (rho * (S - u)**2 - Bx**2) / (rho * (S - u) * (S - SM) - Bx**2)
    Bz1 = Bz * (rho * (S - u)**2 - Bx**2) / (rho * (S - u) * (S - SM) - Bx**2)
    e1 = ((S - u) * e - pT * u + pT1 * SM + Bx * ((u*Bx + v*By + w*Bz) - (u1*Bx + v1*By1 + w1*Bz1))) / (S - SM)
    
    return rho1, u1, v1, w1, By1, Bz1, e1


def get_U_parameters_2(rho1_L, rho1_R, u1_L, u1_R, v1_L, v1_R, w1_L, w1_R, Bx, By1_L, By1_R, Bz1_L, Bz1_R, e1_L, e1_R, SM):
    u2 = SM
    v2 = (np.sqrt(rho1_L) * v1_L + np.sqrt(rho1_R) * v1_R + (By1_R - By1_L) * np.sign(Bx)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    w2 = (np.sqrt(rho1_L) * w1_L + np.sqrt(rho1_R) * w1_R + (Bz1_R - Bz1_L) * np.sign(Bx)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    By2 = (np.sqrt(rho1_L) * By1_R + np.sqrt(rho1_R) * By1_L + np.sqrt(rho1_L * rho1_R) * (v1_R - v1_L) * np.sign(Bx)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    Bz2 = (np.sqrt(rho1_L) * Bz1_R + np.sqrt(rho1_R) * Bz1_L + np.sqrt(rho1_L * rho1_R) * (w1_R - w1_L) * np.sign(Bx)) / (np.sqrt(rho1_L) + np.sqrt(rho1_R))
    e2_L = e1_L - np.sqrt(rho1_L) * ((u1_L * Bx + v1_L * By1_L + w1_L * Bz1_L) - (u2 * Bx + v2 * By2 + w2 * Bz2)) * np.sign(Bx)
    e2_R = e1_R + np.sqrt(rho1_R) * ((u1_R * Bx + v1_R * By1_R + w1_R * Bz1_R) - (u2 * Bx + v2 * By2 + w2 * Bz2)) * np.sign(Bx)

    return u2, v2, w2, By2, Bz2, e2_L, e2_R


def get_flux(rho, u, v, w, Bx, By, Bz, e, pT, F):
    F[0, :] = rho * u
    F[1, :] = rho * u**2 + pT - Bx**2
    F[2, :] = rho * u * v - Bx * By
    F[3, :] = rho * u * w - Bx * Bz
    F[4, :] = 0.0
    F[5, :] = u * By - v * Bx
    F[6, :] = u * Bz - w * Bx
    F[7, :] = (e + pT) * u - Bx * (Bx * u + By * v + Bz * w)
    return F


def get_flux_HLLD(rho, u, v, w, Bx, By, Bz, e, gamma, F, axis):

    p = (gamma - 1.0) * (e - rho * 0.5 * (u**2 + v**2 + w**2) - 0.5 * (Bx**2 + By**2 + Bz**2))
    # 半整数格子点上のものを使わないといけない。1次元だと関係なかったけど。
    Bx_half = 0.5 * (Bx + np.roll(Bx, -1, axis=axis))
    rho_L, u_L, v_L, w_L, By_L, Bz_L, p_L = get_L_component(rho, u, v, w, By, Bz, p, axis)
    rho_R, u_R, v_R, w_R, By_R, Bz_R, p_R = get_R_component(rho, u, v, w, By, Bz, p, axis)
    pT_L = p_L + 0.5 * (Bx_half**2 + By_L**2 + Bz_L**2)
    pT_R = p_R + 0.5 * (Bx_half**2 + By_R**2 + Bz_R**2)
    e_L = p_L / (gamma - 1.0) + rho_L * 0.5 * (u_L**2 + v_L**2 + w_L**2) + 0.5 * (Bx_half**2 + By_L**2 + Bz_L**2)
    e_R = p_R / (gamma - 1.0) + rho_R * 0.5 * (u_R**2 + v_R**2 + w_R**2) + 0.5 * (Bx_half**2 + By_R**2 + Bz_R**2)
    cs_L = np.sqrt(gamma * p_L / rho_L)
    cs_R = np.sqrt(gamma * p_R / rho_R)
    ca_L = np.sqrt((Bx_half**2 + By_L**2 + Bz_L**2) / rho_L)
    ca_R = np.sqrt((Bx_half**2 + By_R**2 + Bz_R**2) / rho_R)
    va_L = np.sqrt(Bx_half**2 / rho_L)
    va_R = np.sqrt(Bx_half**2 / rho_R)
    cf_L = np.sqrt(0.5 * (cs_L**2 + ca_L**2 + np.sqrt((cs_L**2 + ca_L**2)**2 - 4.0 * cs_L**2 * va_L**2)))
    cf_R = np.sqrt(0.5 * (cs_R**2 + ca_R**2 + np.sqrt((cs_R**2 + ca_R**2)**2 - 4.0 * cs_R**2 * va_R**2)))
    S_L = np.minimum(u_L - cf_L, u_R - cf_R)
    S_R = np.maximum(u_L + cf_L, u_R + cf_R)
    #S_L = np.minimum(S_L, 0.0)
    #S_R = np.maximum(S_R, 0.0)

    SM = ((S_R - u_R) * rho_R * u_R - (S_L - u_L) * rho_L * u_L - pT_R + pT_L) / ((S_R - u_R) * rho_R - (S_L - u_L) * rho_L)
    pT1 = ((S_R - u_R) * rho_R * pT_L - (S_L - u_L) * rho_L * pT_R + rho_L * rho_R * (S_R - u_R) * (S_L - u_L) * (u_R - u_L)) / ((S_R - u_R) * rho_R - (S_L - u_L) * rho_L)
    pT1_L = pT1
    pT1_R = pT1
    rho1_L, u1_L, v1_L, w1_L, By1_L, Bz1_L, e1_L = get_U_parameters_1(rho_L, u_L, v_L, w_L, Bx_half, By_L, Bz_L, e_L, pT_L, pT1_L, S_L, SM)
    rho1_R, u1_R, v1_R, w1_R, By1_R, Bz1_R, e1_R = get_U_parameters_1(rho_R, u_R, v_R, w_R, Bx_half, By_R, Bz_R, e_R, pT_R, pT1_R, S_R, SM)

    S1_L = SM - np.sqrt(Bx_half**2 / rho1_L)
    S1_R = SM + np.sqrt(Bx_half**2 / rho1_R)

    u2, v2, w2, By2, Bz2, e2_L, e2_R = get_U_parameters_2(rho1_L, rho1_R, u1_L, u1_R, v1_L, v1_R, w1_L, w1_R, Bx_half, By1_L, By1_R, Bz1_L, Bz1_R, e1_L, e1_R, SM)
    pT2_L = pT1
    pT2_R = pT1
    rho2_L = rho1_L
    rho2_R = rho1_R

    # Fの選択
    F_L = np.zeros(F.shape)
    F1_L = np.zeros(F.shape)
    F2_L = np.zeros(F.shape)
    F_R = np.zeros(F.shape)
    F1_R = np.zeros(F.shape)
    F2_R = np.zeros(F.shape)
    F_L = get_flux(rho_L, u_L, v_L, w_L, Bx_half, By_L, Bz_L, e_L, pT_L, F_L)
    F1_L = get_flux(rho1_L, u1_L, v1_L, w1_L, Bx_half, By1_L, Bz1_L, e1_L, pT1_L, F1_L)
    F2_L = get_flux(rho2_L, u2, v2, w2, Bx_half, By2, Bz2, e2_L, pT2_L, F2_L)
    F_R = get_flux(rho_R, u_R, v_R, w_R, Bx_half, By_R, Bz_R, e_R, pT_R, F_R)
    F1_R = get_flux(rho1_R, u1_R, v1_R, w1_R, Bx_half, By1_R, Bz1_R, e1_R, pT1_R, F1_R)
    F2_R = get_flux(rho2_R, u2, v2, w2, Bx_half, By2, Bz2, e2_R, pT2_R, F2_R)

    F = F_L * (S_L > 0.0) + F1_L * ((S_L <= 0.0) & (0.0 < S1_L)) + F2_L * ((S1_L <= 0.0) & (0.0 < SM)) \
      + F_R * (S_R <= 0.0) + F1_R * ((S1_R <= 0.0) & (0.0 < S_R)) + F2_R * ((SM <= 0.0) & (0.0 < S1_R))

    return F    


def boundary_symmetric_x(U):
    
    U[:, 0] = U[:, 1]
    U[:, -1] = U[:, -2]

    return U

接続領域

In [4]:
def send_MHD_to_PIC():
    pass 


def send_PIC_to_MHD():
    pass

Alfven波の伝搬

PIC初期化

In [5]:
c = 0.5
epsilon0 = 1.0
mu_0 = 1.0 / (epsilon0 * c**2)
m_unit = 1.0
r_m = 1/100
m_electron = 1 * m_unit
m_ion = m_electron / r_m
t_r = 1.0
r_q = 1.0
n_e = 10 #ここは手動で調整すること
B0 = np.sqrt(n_e) / 1.0
n_i = int(n_e / r_q)
T_i  = (B0**2 / 2.0 / mu_0) / (n_i + n_e * t_r)
T_e = T_i * t_r
q_unit = np.sqrt(epsilon0 * T_e / n_e)
q_electron = -1 * q_unit
q_ion = r_q * q_unit
debye_length = np.sqrt(epsilon0 * T_e / n_e / q_electron**2)
omega_pe = np.sqrt(n_e * q_electron**2 / m_electron / epsilon0)
omega_pi = np.sqrt(n_i * q_ion**2 / m_ion / epsilon0)
omega_ce = q_electron * B0 / m_electron
omega_ci = q_ion * B0 / m_ion
ion_inertial_length = c / omega_pi
electron_inertial_length = c / omega_pe
v_ion = np.array([0.0, 0.0, 0.0])
v_electron = np.array([0.0, 0.0, 0.0])
v_thermal_electron = np.sqrt(T_e / m_electron)
v_thermal_ion = np.sqrt(T_i / m_ion)

dx_pic = 1.0
nx_pic = int(electron_inertial_length * 40)
x_max_pic = nx_pic * dx_pic
x_coordinate = np.arange(0.0, x_max_pic, dx_pic)
dt_pic = 1.0
step = 20000
t_max = step * dt_pic


E_pic = np.zeros([3, nx_pic])
B_pic = np.zeros([3, nx_pic])
current_pic = np.zeros([3, nx_pic])

n_ion = int(n_i * nx_pic)
n_electron = int(n_ion * abs(q_ion / q_electron))
x_pic = np.zeros([3, n_ion + n_electron])
v_pic = np.zeros([3, n_ion + n_electron])

np.random.RandomState(1)
x_start_ion = np.random.rand(n_ion) * x_max_pic
x_start_electron = np.random.rand(n_electron) * x_max_pic

x_pic[0, :] = np.concatenate([x_start_ion, x_start_electron])
v_pic[0, :n_ion] = np.asarray(stats.norm.rvs(v_ion[0], v_thermal_ion, size=n_ion))
v_pic[0, n_ion:] = np.asarray(stats.norm.rvs(v_electron[0], v_thermal_electron, size=n_electron))
v_pic[1, :n_ion] = np.asarray(stats.norm.rvs(v_ion[1], v_thermal_ion, size=n_ion))
v_pic[1, n_ion:] = np.asarray(stats.norm.rvs(v_electron[1], v_thermal_electron, size=n_electron))
v_pic[2, :n_ion] = np.asarray(stats.norm.rvs(v_ion[2], v_thermal_ion, size=n_ion))
v_pic[2, n_ion:] = np.asarray(stats.norm.rvs(v_electron[2], v_thermal_electron, size=n_electron))

q_list = np.zeros(n_ion + n_electron)
q_list[:n_ion] = q_ion
q_list[n_ion:] = q_electron
m_list = np.zeros(n_ion + n_electron)
m_list[:n_ion] = m_ion
m_list[n_ion:] = m_electron

print(f"total number of particles is {n_ion + n_electron}.")
print(f"Box size is {nx_pic}")

total number of particles is 1600.
Box size is 80


MHD初期化 \
U1は上半分、U2は下半分のつもり

In [7]:
gamma = 5.0/3.0
B0 = 1.0
rho0 = 1.0
beta = 1.0
p0 = beta * B0**2 / 2.0
dx_mhd = 1.0
dy_mhd = 1.0

nx_mhd = nx_pic
x_max_mhd = nx_mhd * dx_mhd
dt_mhd = 0.0
CFL = 0.4
x_coordinate = np.arange(0.0, x_max_mhd, dx_mhd)

U1 = np.zeros([8, nx_mhd])
rho_init = rho0
u_init = 0.0
v_init = 0.0
w_init = 0.0
Bx_init = B0
By_init = 0.0
Bz_init = 0.0
p_init = beta * (Bx_init**2 + By_init**2 + Bz_init**2) / 2.0
U1[0, :] = rho_init
U1[1, :] = rho_init * u_init
U1[2, :] = rho_init * v_init
U1[3, :] = rho_init * w_init
U1[4, :] = Bx_init
U1[5, :] = By_init
U1[6, :] = Bz_init
U1[7, :] = p_init/(gamma-1) + rho_init * (u_init**2 + v_init**2 + w_init**2)/2 + (Bx_init**2 + By_init**2 + Bz_init**2)/2

F1 = np.zeros(U1.shape)
F_bar1 = np.zeros(F1.shape)
G1 = np.zeros(U1.shape)
G_bar1 = np.zeros(G1.shape)

U2 = U1.copy()
F2 = F1.copy()
F_bar2 = F_bar1.copy()
G2 = G1.copy()
G_bar2 = G_bar1.copy()

print(f"U size is {U1.shape}")

U size is (8, 80)


interface用の変数など

時間発展

In [ ]:
#STEP1:PICとMHDの時間を合わせる
#以下、t1からt2に時間発展させるとする
#ただし、t2 = t1 + dt_mhdとする

time = 0.0
for k in range(step+1):

    #STEP2:MHDのデータをPICに送る
    
    #STEP3:PIC数ステップの平均を取る
    # PIC
    #-----------------------------------------------------------
    dt_pic = dt_mhd / np.sqrt(r_m) 

    B_pic = time_evolution_B(E_pic, dx_pic, dy_pic, dt_pic/2, B_pic) 

    v_pic = time_evolution_v(c, E_pic, B_pic, x_pic, q_list, m_list, nx_pic, ny_pic, dx_pic, dy_pic, dt_pic, v_pic)

    x_pic = time_evolution_x(c, dt_pic/2, v_pic, x_pic)
    v_pic, x_pic = refrective_condition_x(v_pic, x_pic, y_max_pic)

    current_pic = get_current_density(c, q_list, v_pic, x_pic, nx_pic, ny_pic, dx_pic, dy_pic, current_pic)

    B_pic = time_evolution_B(E_pic, dx_pic, dy_pic, dt_pic/2, B_pic) 

    E_pic = time_evolution_E(B_pic, current_pic, c, epsilon0, dx_pic, dy_pic, dt_pic, E_pic)
    rho_pic = get_rho(q_list, x_pic, nx_pic, ny_pic, dx_pic, dy_pic)

    x_pic = time_evolution_x(c, dt_pic/2, v_pic, x_pic)
    v_pic, x_pic = refrective_condition_x(v_pic, x_pic, y_max_pic)
    #-----------------------------------------------------------

    #STEP4:STEP3のデータをMHDに送る

    #STEP5:MHDを1ステップ進める(dt_mhdでt1からt2)
    # MHD
    #-----------------------------------------------------------
    
    if np.isnan(time):
        print(f"{k} steps (t = {time:.3f}) : Calculation is crashed!")
        break
    
    U_bar = U.copy()

    rho_mhd = U[0, :]
    u_mhd = U[1, :] / rho_mhd
    v_mhd = U[2, :] / rho_mhd
    w_mhd = U[3, :] / rho_mhd
    Bx_mhd = U[4, :]
    By_mhd = U[5, :]
    Bz_mhd = U[6, :]
    e_mhd = U[7, :]
    p_mhd = (gamma - 1.0) \
          * (e_mhd - 0.5 * rho_mhd * (u_mhd**2+v_mhd**2+w_mhd**2)
              - 0.5 * (Bx_mhd**2+By_mhd**2+Bz_mhd**2))

    F = get_flux_HLLD(rho_mhd, u_mhd, v_mhd, w_mhd, Bx_mhd, By_mhd, Bz_mhd, e_mhd, gamma, F, axis=0)

    cs_mhd = np.sqrt(gamma * p_mhd / rho_mhd)
    ca_mhd = np.sqrt((Bx_mhd**2 + By_mhd**2 + Bz_mhd**2) / rho_mhd)
    dt_mhd = CFL * np.min(1.0 / ((np.abs(u_mhd) + np.sqrt(cs_mhd**2 + ca_mhd**2)) / dx_mhd
                                  + (np.abs(v_mhd) + np.sqrt(cs_mhd**2 + ca_mhd**2)) / dy_mhd))
    
    U_bar += -dt_mhd/dx_mhd * (F - np.roll(F, 1, axis=1))
    U_bar = boundary_symmetric_x(U_bar)

    rho_mhd = U[0, :]
    u_mhd = U[1, :] / rho_mhd
    v_mhd = U[2, :] / rho_mhd
    w_mhd = U[3, :] / rho_mhd
    Bx_mhd = U[4, :]
    By_mhd = U[5, :]
    Bz_mhd = U[6, :]
    e_mhd = U[7, :]
    p_mhd = (gamma - 1.0) \
          * (e_mhd - 0.5 * rho_mhd * (u_mhd**2+v_mhd**2+w_mhd**2)
              - 0.5 * (Bx_mhd**2+By_mhd**2+Bz_mhd**2))
    
    F_bar = get_flux_HLLD(rho_mhd, u_mhd, v_mhd, w_mhd, Bx_mhd, By_mhd, Bz_mhd, e_mhd, gamma, F_bar, axis=0)
    
    F = 0.5 * (F + F_bar)

    U += -dt_mhd/dx_mhd * (F - np.roll(F, 1, axis=1))
    # y方向自由境界
    U = boundary_symmetric_x(U)

    time += dt_mhd
    #-----------------------------------------------------------

    #STEP6:PICの時間をt1に戻す

    #STEP7:PICをMHD1ステップ分進める
    #境界条件はMHDのt1とt2から線形内挿
    

    # save
    if (step % 100 == 0):
        print(f"{step} step done... : time = {time:.5f}, dt = {dt_mhd:.5f}")